In [1]:
import pandas as pd
import sqlite3
import os
import json
import glob
import re
import pandas as pd

In [2]:
db_file=r'/home/yoba/Projects/education4climate/open-data/output/education4climate.sqlite'
db=sqlite3.connect(db_file)

In [3]:
courses=pd.read_sql('''
    select distinct a.year, 
                    a.id_course, 
                    a.name, 
                    a.activity, 
                    a.content, 
                    a.goal, 
                    b.languages
    from T_courses a, ( select id_course, languages, year from R_courses_languages ) b
    where coalesce(activity, content, goal) <> '' 
            and a.id_course=b.id_course 
            and a.year=b.year
            and a.year=2021
            and b.languages='fr'
''',db)

In [4]:
courses.sample(5).head()

,year,id_course,name,activity,content,goal,languages
8991,2021,DROI-S600,Initiation au droit des affaires,Exposé de la théorie et exemples issus de la p...,Le cours est composé des chapitres suivants: ...,Donner les bases essentielles du droit des aff...,fr
5694,2021,21441,"UE8 Q1 Langue étrangère 1 Anglais : outils, do...",Outils \r\n- Révision de points grammaticaux...,Outils : approfondissement de la théorie et de...,Développer et améliorer la capacité d’utilisat...,fr
8968,2021,DROI-D103,"Introduction au droit, y compris les aspects s...",Le cours compte 5 crédits qui se répartissent ...,Une première partie s'attache à une étude anal...,Le cours entend vous à amener à : à d...,fr
1396,2021,2BEEII12Q,Expression écrite II,Le cours est dispensé une semaine sur deux ; i...,Cette unité d’enseignement permet aux étudiant...,"Au terme de l’unité d’enseignement, l’étudiant",fr
11056,2021,KDBG0001-1,"Revalidation gériatrique, Revalidation gériatr...",Cette activité d'apprentissage se base sur un ...,Au terme de l'activité d'apprentissage de reva...,Au terme de l'activité d'apprentissage de reva...,fr


https://scikit-learn.org/stable/auto_examples/text/plot_hashing_vs_dict_vectorizer.html#sphx-glr-auto-examples-text-plot-hashing-vs-dict-vectorizer-py

In [5]:
def tokenize(doc):
    """Extract tokens from doc.

    This uses a simple regex that matches word characters to break strings
    into tokens. For a more principled approach, see CountVectorizer or
    TfidfVectorizer.
    """
    return list(tok for tok in re.findall(r"\w+", doc.lower()) if len(tok)>2)

tokenize("This is a simple example, isn't it?") 

['this', 'simple', 'example', 'isn']

Breaking a text document into word tokens, potentially losing the order information between the words in a sentence is often called a *Bag of Words* representation.

In [6]:
tk={}
for row in courses.itertuples():
    tk[row.id_course]=tokenize(' '.join([row.name,row.activity,row.content,row.goal]))

In [7]:
tk['MECA0524-1']

['cad',
 'geometric',
 'algorithms',
 'cours',
 'cathedra',
 'séances',
 'initiation',
 'programmation',
 'géométrique',
 'projet',
 'programmation',
 'conséquent',
 'modèles',
 'repésentation',
 'cao',
 'courbes',
 'surfaces',
 'paramétriques',
 'modèles',
 'mathématique',
 'interprétation',
 'géométrique',
 'des',
 'courbes',
 'surfaces',
 'bézier',
 'splines',
 'problèmes',
 'associés',
 'géométrie',
 'algorithmique',
 'enveloppe',
 'convexe',
 'localisation',
 'diagrammes',
 'voronoï',
 'triangulations',
 'delaunay',
 'maillages',
 'résolution',
 'problèmes',
 'simples',
 'cao',
 'par',
 'programmation',
 'dans',
 'environnement',
 'dédié',
 'connaitre',
 'les',
 'modèles',
 'représentation',
 'utilisés',
 'dans',
 'les',
 'logiciels',
 'cao',
 'connaitre',
 'les',
 'outils',
 'géométrie',
 'algorithmique',
 'applicables',
 'aux',
 'modèles',
 'cao',
 'programmer',
 'dans',
 'environnement',
 'graphique']

In [8]:
from collections import defaultdict


def token_freqs(doc):
    """Extract a dict mapping tokens from doc to their occurrences."""

    freq = defaultdict(int)
    for tok in tokenize(doc):
        freq[tok] += 1
    return freq


token_freqs("That is one example, but this is another one")

defaultdict(int,
            {'that': 1,
             'one': 2,
             'example': 1,
             'but': 1,
             'this': 1,
             'another': 1})

In [9]:
tf={}
for row in courses.itertuples():
    tf[row.id_course]=token_freqs(' '.join([row.name,row.activity,row.content,row.goal]))

In [10]:
tf['MECA0524-1']

defaultdict(int,
            {'cad': 1,
             'geometric': 1,
             'algorithms': 1,
             'cours': 1,
             'cathedra': 1,
             'séances': 1,
             'initiation': 1,
             'programmation': 3,
             'géométrique': 2,
             'projet': 1,
             'conséquent': 1,
             'modèles': 4,
             'repésentation': 1,
             'cao': 4,
             'courbes': 2,
             'surfaces': 2,
             'paramétriques': 1,
             'mathématique': 1,
             'interprétation': 1,
             'des': 1,
             'bézier': 1,
             'splines': 1,
             'problèmes': 2,
             'associés': 1,
             'géométrie': 2,
             'algorithmique': 2,
             'enveloppe': 1,
             'convexe': 1,
             'localisation': 1,
             'diagrammes': 1,
             'voronoï': 1,
             'triangulations': 1,
             'delaunay': 1,
             'maillages': 1,
   

In [11]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
v.fit(D)

DictVectorizer(sparse=False)

In [12]:
v.transform({'foo': 4, 'unseen_feature': 3})

array([[0., 0., 4.]])

In [13]:
v.inverse_transform(v.transform({'foo': 4, 'unseen_feature': 3}))

[{'foo': 4.0}]

In [14]:
v.fit(tf.values())

DictVectorizer(sparse=False)

In [15]:
v.inverse_transform(v.transform({'sténo': 4, 'dactylo':4,'mécanique': 3,'astrophysique':5}))
# sténo et dactylo ne sont pas dans le corpus de documents

[{'astrophysique': 5.0, 'mécanique': 3.0}]

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

'''
ignoring terms that appear in more than 50% of the documents (as set by max_df=0.5) 
and terms that are not present in at least 5 documents (set by min_df=5)
'''

vectorizer = TfidfVectorizer(
    max_df=0.1,
    min_df=10,
    lowercase=True,
)

X_tfidf = vectorizer.fit_transform([' '.join([row.name,row.activity,row.content,row.goal]) for row in courses.itertuples()])

In [17]:
courses.shape

(11658, 7)

In [18]:
X_tfidf.shape

(11658, 11890)

In [19]:
X_tfidf.size

1048837

https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py

In [20]:
from sklearn.cluster import KMeans
import numpy as np

ms_countOf_clusters=20
ms_countOf_models=1

for seed in range(ms_countOf_models):
    kmeans = KMeans(
        n_clusters=ms_countOf_clusters,
        max_iter=50,
        n_init=10,
        random_state=seed,
    ).fit(X_tfidf)
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    #cluster_sizes.sort()
    print(f"{cluster_sizes}")


[ 243  463  313  157  183  341  707  464  320  199  544 4234  665  468
  129  657  674  444  313  140]


In [21]:
original_space_centroids = kmeans.cluster_centers_
order_centroids = original_space_centroids.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

ms_countOf_words=15

for i in range(ms_countOf_clusters):
    print(f"Cluster {i}({cluster_sizes[i]}): ", end="")
    for ind in order_centroids[i, :ms_countOf_words]:
        print(f"{terms[ind]} ", end="")
    print()

Cluster 0(243): chimie chimiques organique laboratoire chimique réactions analytique cinétique propriétés molécules organiques dosage laboratoires pharmaceutique biochimie 
Cluster 1(463): santé soins infirmiers patient clinique promotion publique soin interventions personne éthique compétence infirmière aat patients 
Cluster 2(313): psychologie psychologiques enfant apprentissages théories psychologique sociale adolescence c2 c4 c5 c6 pédagogie pédagogiques pondération 
Cluster 3(157): traduction français interprétation consécutive simultanée vers discours texte néerlandais linguistiques traduire espagnol adapter allemand technologiques 
Cluster 4(183): statistique statistiques régression probabilités tests descriptive variables logiciel inférence paramétriques linéaire corrélation variance estimation test 
Cluster 5(341): art arts architecture culture musicale plastique education expression artistique archéologie initiation artistiques musique didactique atelier 
Cluster 6(707): dida

In [22]:
len(vectorizer.get_feature_names_out())

11890

In [23]:
X_tfidf.shape

(11658, 11890)

In [24]:
vectorizer.inverse_transform(X_tfidf[kmeans.labels_==4,])

[array(['bloc2', 'suffisante', 'probabiliste', 'quotidiennement',
        'confrontés', 'fiabilité', 'inférentielle', 'générées', 'jacente',
        'conclusions', 'tirer', 'décrivent', 'nombres', 'graphes',
        'perdre', 'représentant', 'simplifier', 'faudra', 'inférence',
        'appelée', 'corrélation', 'covariance', 'marginales',
        'mathématique', 'espérance', 'aléatoires', 'exponentielle',
        'continu', 'géométrique', 'uniforme', 'binomiale', 'bernoulli',
        'discret', 'conditionnelles', 'fréquentielle', 'linéaire',
        'dispersion', 'distributions', 'redécouvrir', 'local',
        'amélioration', 'reposent', 'chances', 'recommandée', 'emblée',
        'complètent', 'effort', 'probabilités', 'statistique', 'variables',
        'profiter', 'pleinement', 'tenir', 'normale', 'couple',
        'facilement', 'reprographie', 'manuel', 'aisance', 'préliminaire',
        'inductive', 'initiation', 'fonctions', 'dégager', 'étape',
        'simples', 'manipulation',

In [25]:
courses.content[kmeans.labels_==4]

104      Syllabus complet disponible au service de repr...
114      Rappel sur les probabilités : 1) Les variables...
124      Introduction :  Révision de statistique approf...
153      Ce cours reprend tout d'abord des rappels des ...
201      1 Introduction : "une saine méfiance statistiq...
                               ...                        
11323    L'UE se compose des activités d'apprentissage,...
11352    Le cours de statistique comprend:\n \n\n 1) Le...
11551    Informatique de gestion 2\nInformatique - Le t...
11552    Voir fiche AA\nInformatique de gestion 1\nPour...
11637    Informatique de gestion\nLe tableur  :\n \n\n ...
Name: content, Length: 183, dtype: object

In [28]:
# Trouvez les mots les plus associés au mot clé 
mot='climatique'
most_associated_words = np.argsort(X_tfidf[:, vectorizer.get_feature_names().index(mot)].todense(),axis=0)[-100:]
features=vectorizer.get_feature_names()

# Imprimez les mots les plus associés
for word in most_associated_words:
    lineNumber=word[0,0]
    line=X_tfidf[lineNumber,:].todense()
    greatestTfIdfFoundInLine=np.argmax(line)
    greatestsTfIdfFoundInLine=np.argsort(line).tolist()[0]
    print(features[greatestTfIdfFoundInLine])
    print([features[greatestsTfIdfFoundInLine[-i]] for i in range(10)])

supérieur
['00', 'supérieur', 'participeront', 'colloques', 'thématique', 'pédagogie', 'témoignages', 'choisie', 'professionnels', 'fiche']
supérieur
['00', 'supérieur', 'pédagogie', 'association', 'séance', 'visés', 'trente', 'accompagneront', 'champ', 'tentent']
régression
['00', 'régression', 'fichiers', 'linéaire', 'internationales', 'statistiques', 'sas', 'pisa', 'ocde', 'amplitude']
indices
['00', 'indices', 'statistiques', 'inférentielles', 'éducation', 'analyses', 'descriptives', 'tendance', 'dispersion', 'régression']
séminaire
['00', 'séminaire', 'entrevues', 'représentations', 'visant', 'problématiques', 'échanger', 'stages', 'éducation', 'mêmes']
vous
['00', 'vous', 'votre', 'transfert', 'devez', 'pédagogique', 'participé', 'opérés', 'catalogue', 'varient']
dispositif
['00', 'dispositif', 'conçu', 'démarches', 'oeuvre', 'didactique', 'final', 'aptitudes', 'supérieur', 'individuels']
éducatif
['00', 'éducatif', 'fédération', 'pédagogiques', 'discours', 'wallonie', 'elles', '

In [29]:
vectorizer.inverse_transform(vectorizer.transform(['physique réchauffement climatique physique']))

[array(['réchauffement', 'physique', 'climatique'], dtype='<U25')]

In [30]:
from sklearn.metrics.pairwise import cosine_distances

In [35]:
vectorizer.inverse_transform(X_tfidf[1567,:])

[array(['baliser', 'pharmacie', 'outcomes', 'listing', 'dentisterie',
        'feuillets', 'topographique', 'kinésithérapie', 'embryologie',
        'ostéologie', 'circulatoire', 'myologie', 'endocrinien',
        'tissulaires', 'digestif', 'locomoteur', 'anatomique', 'appareil',
        'histologie', 'immunitaire', 'nerveux', 'médecins', 'urinaire',
        'médecine', 'démonstrations', 'médicale', 'anatomie',
        'respiratoire', 'physiologiques', 'exhaustif', 'sélectionner',
        'déroulent', 'spécialisée', 'cotée', 'biologiques', 'anatomiques',
        'central', 'motricité', 'téléchargeable', 'physique',
        'orientation', 'utile', 'descriptives', 'répartis', 'humaine',
        'sections', 'destiné', 'devra', 'description', 'langage',
        'précises', 'biomédicales', 'comporte', 'précise', 'corps',
        'systématique', 'cursus', 'proposés', 'acquisition',
        'essentielles', 'démonstration', 'générales', 'cet', 'portée',
        'donne', 'préciser', 'aperçu', '

In [41]:
cosine_distances(X_tfidf[1567,:],vectorizer.transform(['pharmacie kinésithérapie embryologie biologiques','médecins biomédicales organes motricité']))

array([[0.84872994, 0.78785414]])